# GeoAlchemy2 ORM Test -- Import CSDs

Source: [https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html](https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html)

In [ ]:
# read configuration
import psycopg2, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

#conn_string = "host=%(SERVER)s user=%(USERNAME)s dbname=%(DATABASE)s password=%(PASSWORD)s sslmode=%(SSLMODE)s" % config
#conn = psycopg2.connect(conn_string) 
#print("Connection established")


# Some psycopg2 intialization

from register_adapters import register_adapters

register_adapters()

## Connect to the DB

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://%(USERNAME)s:%(PASSWORD)s@%(SERVER)s/%(DATABASE)s?sslmode=%(SSLMODE)s" % config,
    client_encoding="utf8", 
    echo=True
)

## Declare a Mapping

We will try to map the CSD digital boundary files 

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.path.abspath(""), "models"))

from models import CensusSubDivision


## Create the Table in the Database

In [ ]:
try:
    CensusSubDivision.__table__.drop(engine)
except:
    pass

CensusSubDivision.__table__.create(engine)

In [ ]:
import geopandas as gpd
import os

# import the 2021 Digital Boundary File
csd_dbf = gpd.read_file(os.path.join(os.path.abspath(""), "data", "lcsd000a21a_e.zip"))

# project to lat/lon

csd_dbf = csd_dbf.to_crs("epsg:4326")

csd_dbf["CSDUID"] = csd_dbf["CSDUID"].astype(int) 
csd_dbf["PRUID"]  = csd_dbf["PRUID"].astype(int)
csd_dbf["LANDAREA"]  = csd_dbf["LANDAREA"].astype(float)

csd_dbf

In [ ]:
csd_dbf.info()

## Create a Session

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()


In [ ]:
# helper function to create table entries from GeoDataFrame
def insert_csd(session, x):
    row = CensusSubDivision( DGUID    = x["DGUID"],     
                CSDUID   = x["CSDUID"],
                name  = x["CSDNAME"], 
                type  = x["CSDTYPE"],
                geometry = x["geometry"].wkt
                )

    session.add(row)


csd_dbf.apply(lambda x: insert_csd(session, x), axis=1)

# commit changes
session.commit()

## Query

In [ ]:
# Get all reserve CSDs 
csd_list = session.query(CensusSubDivision).order_by(CensusSubDivision.name).where(CensusSubDivision.type=="IRI").all()

for i in csd_list:
    print(i.name)
